In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
import os
import shutil

# Définir les chemins des dossiers
base_dir = r'C:\Users\Aerlaw\Pictures\IA'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'champignon_test')

# Créer des sous-dossiers pour les classes comestible et toxique dans le dossier train
train_comestible_dir = os.path.join(base_dir, 'train_comestible')
train_toxique_dir = os.path.join(base_dir, 'train_toxique')

os.makedirs(train_comestible_dir, exist_ok=True)
os.makedirs(train_toxique_dir, exist_ok=True)

# Distribuer les images dans les sous-dossiers respectifs
for filename in os.listdir(train_dir):
    if filename.startswith('comestible'):
        shutil.copy(os.path.join(train_dir, filename), os.path.join(train_comestible_dir, filename))
    elif filename.startswith('toxique'):
        shutil.copy(os.path.join(train_dir, filename), os.path.join(train_toxique_dir, filename))

# Utilisation d'ImageDataGenerator pour la préparation des données
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Définir le modèle CNN
model = Sequential([
    tf.keras.Input(shape=(128, 128, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

# Sauvegarder le modèle
model.save('mushroom_classifier.keras')

# Charger le modèle
model = tf.keras.models.load_model('mushroom_classifier.keras')

# Préparation des données de test
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode=None,  # Pas besoin des étiquettes pour la prédiction
    shuffle=False
)

# Faire des prédictions sur les images de test
predictions = model.predict(test_generator)
predicted_classes = np.where(predictions > 0.5, 'Toxique', 'Comestible')

# Afficher les résultats
for filename, pred_class in zip(test_generator.filenames, predicted_classes):
    print(f'Image: {filename} - Prédiction: {pred_class}')


Found 528 images belonging to 4 classes.
Found 130 images belonging to 4 classes.
Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.3958 - loss: -561.7024 - val_accuracy: 0.5000 - val_loss: -16021.0928
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.4634 - loss: -66812.6094 - val_accuracy: 0.5000 - val_loss: -709757.9375
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.4939 - loss: -1618476.8750 - val_accuracy: 0.5000 - val_loss: -9303899.0000
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.4840 - loss: -16024226.0000 - val_accuracy: 0.5000 - val_loss: -63609296.0000
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.4835 - loss: -93967144.0000 - val_accuracy: 0.5000 - val_loss: -291028800.0000
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.4972 - loss: -393870944.0000 - val_accuracy: 0.5000 - val_loss: -1002832128.0000
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.4745 - loss: 

ValueError: Must provide at least one structure

# Test n°2

In [23]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

# Chemins des répertoires
train_dir = r'C:\Users\Aerlaw\Pictures\IA\train'
validation_dir = r'C:\Users\Aerlaw\Pictures\IA\validation'
test_images_dir = r'C:\Users\Aerlaw\Pictures\IA\champignon_test'

# Paramètres
img_width, img_height = 150, 150
batch_size = 32
epochs = 25

# Vérification des répertoires
if not os.path.isdir(train_dir):
    raise ValueError(f"Le répertoire d'entraînement {train_dir} n'existe pas.")
if not os.path.isdir(validation_dir):
    raise ValueError(f"Le répertoire de validation {validation_dir} n'existe pas.")
if not os.path.isdir(test_images_dir):
    raise ValueError(f"Le répertoire de test {test_images_dir} n'existe pas.")

# Préparation des générateurs de données
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['comestible', 'toxique']
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['comestible', 'toxique'],
    shuffle=False
)

# Construction du modèle CNN
model = Sequential([
    Input(shape=(img_width, img_height, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entraînement du modèle
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Évaluation du modèle
loss, accuracy = model.evaluate(validation_generator)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Prédiction sur de nouvelles images
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    prediction = model.predict(img_array)
    
    if prediction < 0.5:
        return 'Comestible'
    else:
        return 'Toxique'

# Exemple d'utilisation
for file in os.listdir(test_images_dir):
    img_path = os.path.join(test_images_dir, file)
    if os.path.isfile(img_path):
        prediction = predict_image(img_path)
        print(f'{file}: {prediction}')


Found 195 images belonging to 2 classes.
Found 130 images belonging to 2 classes.
Epoch 1/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.5054 - loss: 0.7101 - val_accuracy: 0.6172 - val_loss: 0.6887
Epoch 2/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.6875 - loss: 0.6144 - val_accuracy: 0.0000e+00 - val_loss: 1.2212


C:\Users\Aerlaw\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Epoch 3/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5900 - loss: 0.7318 - val_accuracy: 0.6172 - val_loss: 0.6664
Epoch 4/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6250 - loss: 0.6546 - val_accuracy: 0.0000e+00 - val_loss: 0.8679
Epoch 5/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5845 - loss: 0.6743 - val_accuracy: 0.6172 - val_loss: 0.6657
Epoch 6/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5625 - loss: 0.6774 - val_accuracy: 0.0000e+00 - val_loss: 0.9124
Epoch 7/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.6197 - loss: 0.6487 - val_accuracy: 0.6250 - val_loss: 0.6752
Epoch 8/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7188 - loss: 0.6339 - val_accuracy: 0.5000 - val_loss: 0.7218
Epoch 9/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5906 - loss: 0.6647 - val_accuracy: 0.6484 - val_loss: 0.6762
Epoch 10/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5625 - loss: 0.6937 - val_accuracy: 0.5000 - val_loss: 0.

In [25]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

# Chemins des répertoires
train_dir = r'C:\Users\Aerlaw\Pictures\IA\train'
validation_dir = r'C:\Users\Aerlaw\Pictures\IA\validation'
test_images_dir = r'C:\Users\Aerlaw\Pictures\IA\champignon_test'

# Paramètres
img_width, img_height = 150, 150
batch_size = 32
epochs = 25

# Vérification des répertoires
if not os.path.isdir(train_dir):
    raise ValueError(f"Le répertoire d'entraînement {train_dir} n'existe pas.")
if not os.path.isdir(validation_dir):
    raise ValueError(f"Le répertoire de validation {validation_dir} n'existe pas.")
if not os.path.isdir(test_images_dir):
    raise ValueError(f"Le répertoire de test {test_images_dir} n'existe pas.")

# Préparation des générateurs de données
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['comestible', 'toxique']
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['comestible', 'toxique'],
    shuffle=False
)

# Répétition des générateurs
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_width, img_height, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32))
).repeat()

validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_width, img_height, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32))
).repeat()

# Construction du modèle CNN
model = Sequential([
    Input(shape=(img_width, img_height, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entraînement du modèle
model.fit(
    train_dataset,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_dataset,
    validation_steps=validation_generator.samples // batch_size
)

# Évaluation du modèle
loss, accuracy = model.evaluate(validation_dataset, steps=validation_generator.samples // batch_size)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Prédiction sur de nouvelles images
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    prediction = model.predict(img_array)
    
    if prediction < 0.5:
        return 'Comestible'
    else:
        return 'Toxique'

# Exemple d'utilisation
for file in os.listdir(test_images_dir):
    img_path = os.path.join(test_images_dir, file)
    if os.path.isfile(img_path):
        prediction = predict_image(img_path)
        print(f'{file}: {prediction}')


Found 195 images belonging to 2 classes.
Found 130 images belonging to 2 classes.
Epoch 1/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 29s 4s/step - accuracy: 0.5074 - loss: 0.7200 - val_accuracy: 0.3906 - val_loss: 0.7044
Epoch 2/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.4630 - loss: 0.7014 - val_accuracy: 0.8061 - val_loss: 0.5081
Epoch 3/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.6128 - loss: 0.6697 - val_accuracy: 0.6531 - val_loss: 0.6475
Epoch 4/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5758 - loss: 0.6704 - val_accuracy: 0.5408 - val_loss: 0.6903
Epoch 5/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.6299 - loss: 0.6645 - val_accuracy: 0.4796 - val_loss: 0.7119
Epoch 6/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5926 - loss: 0.6644 - val_accuracy: 0.6172 - val_loss: 0.6663
Epoch 7/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5810 - loss: 0.6756 - val_accuracy: 0.8061 - val_loss: 0.5505
Epoch 8/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - a

# Nouveau test

In [28]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

# Chemins des répertoires
train_dir = r'C:\Users\Aerlaw\Pictures\IA\train'
validation_dir = r'C:\Users\Aerlaw\Pictures\IA\validation'
test_images_dir = r'C:\Users\Aerlaw\Pictures\IA\champignon_test'

# Paramètres
img_width, img_height = 150, 150
batch_size = 32
epochs = 25

# Vérification des répertoires
if not os.path.isdir(train_dir):
    raise ValueError(f"Le répertoire d'entraînement {train_dir} n'existe pas.")
if not os.path.isdir(validation_dir):
    raise ValueError(f"Le répertoire de validation {validation_dir} n'existe pas.")
if not os.path.isdir(test_images_dir):
    raise ValueError(f"Le répertoire de test {test_images_dir} n'existe pas.")

# Préparation des générateurs de données
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['comestible', 'toxique']
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['comestible', 'toxique'],
    shuffle=False
)

# Répétition des générateurs
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_width, img_height, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32))
).repeat()

validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_width, img_height, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32))
).repeat()

# Construction du modèle CNN
model = Sequential([
    Input(shape=(img_width, img_height, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entraînement du modèle
model.fit(
    train_dataset,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_dataset,
    validation_steps=validation_generator.samples // batch_size
)

# Évaluation du modèle
loss, accuracy = model.evaluate(validation_dataset, steps=validation_generator.samples // batch_size)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Prédiction sur de nouvelles images
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    prediction = model.predict(img_array)
    
    if prediction < 0.5:
        return 'Comestible'
    else:
        return 'Toxique'

# Exemple d'utilisation
for file in os.listdir(test_images_dir):
    img_path = os.path.join(test_images_dir, file)
    if os.path.isfile(img_path):
        prediction = predict_image(img_path)
        print(f'{file}: {prediction}')


Found 195 images belonging to 2 classes.
Found 130 images belonging to 2 classes.
Epoch 1/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 34s 4s/step - accuracy: 0.5211 - loss: 0.7153 - val_accuracy: 0.6172 - val_loss: 0.6977
Epoch 2/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5914 - loss: 0.6988 - val_accuracy: 0.8061 - val_loss: 0.5831
Epoch 3/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.5585 - loss: 0.6769 - val_accuracy: 0.6531 - val_loss: 0.6351
Epoch 4/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.5369 - loss: 0.7067 - val_accuracy: 0.4796 - val_loss: 0.7235
Epoch 5/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.6676 - loss: 0.6025 - val_accuracy: 0.4898 - val_loss: 0.7287
Epoch 6/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.5895 - loss: 0.6521 - val_accuracy: 0.6641 - val_loss: 0.6597
Epoch 7/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.6213 - loss: 0.6455 - val_accuracy: 0.7959 - val_loss: 0.5923
Epoch 8/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - a